In [34]:
import os
from dotenv import load_dotenv
from azure.ai.textanalytics import TextAnalyticsClient as tac
from azure.core.credentials import AzureKeyCredential


In [41]:
from azure.ai.textanalytics import (
    AnalyzeSentimentAction,
    ExtractKeyPhrasesAction,
    RecognizeEntitiesAction,
)

In [42]:
# Load environment variables
load_dotenv()
key = os.getenv("TEXT_ANALYTICS_KEY")
endpoint = os.getenv("TEXT_ANALYTICS_ENDPOINT")


In [43]:
# Authenticate client
credential = AzureKeyCredential(key)
client = tac(endpoint=endpoint, credential=credential)


In [51]:
# Example: Sentiment Analysis
documents = ["I love Azure Text Analytics! It's fantastic.",
             "good lord!","azure is too hard a name","azure get the job done",
             "azure is meh",
             "three muskersteeres, or three technocrats ha!",
             "oh well,"
             ]

try:
    response = client.analyze_sentiment(documents=documents)
    for doc in response:
        print(f"Sentiment: {doc.sentiment}")
        print(f"Confidence Scores: {doc.confidence_scores}")
except Exception as e:
    print(f"Error: {e}")

Sentiment: positive
Confidence Scores: {'positive': 1.0, 'neutral': 0.0, 'negative': 0.0}
Sentiment: positive
Confidence Scores: {'positive': 0.97, 'neutral': 0.02, 'negative': 0.01}
Sentiment: negative
Confidence Scores: {'positive': 0.0, 'neutral': 0.02, 'negative': 0.98}
Sentiment: positive
Confidence Scores: {'positive': 0.93, 'neutral': 0.07, 'negative': 0.01}
Sentiment: neutral
Confidence Scores: {'positive': 0.05, 'neutral': 0.85, 'negative': 0.1}
Sentiment: neutral
Confidence Scores: {'positive': 0.03, 'neutral': 0.95, 'negative': 0.01}
Sentiment: positive
Confidence Scores: {'positive': 0.48, 'neutral': 0.43, 'negative': 0.09}


In [49]:
# Combined Analysis (Sentiment + Key Phrases + Entities)
documents = ["Maybe we need to choose which Azure project to proceed with by Monday, what is microsoft fabric?! "]

try:
    poller = client.begin_analyze_actions(
        documents=documents,
        actions=[
            AnalyzeSentimentAction(), 
            ExtractKeyPhrasesAction(), 
            RecognizeEntitiesAction(),
        ],
    )
    results = poller.result()

    for result in results:
        for action in result:
            if action.kind == "SentimentAnalysis":
                # Access sentiment directly from action
                print(f"Sentiment: {action.sentiment}")
                print(f"Confidence Scores: {action.confidence_scores}")
            
            elif action.kind == "KeyPhraseExtraction":
                # Access key phrases directly
                print(f"Key Phrases: {action.key_phrases}")
            
            elif action.kind == "EntityRecognition":
                # Access entities directly
                print(f"Entities: {[entity.text for entity in action.entities]}")

except Exception as e:
    print(f"Error: {e}")

Sentiment: neutral
Confidence Scores: {'positive': 0.12, 'neutral': 0.71, 'negative': 0.17}
Key Phrases: ['Azure project', 'microsoft fabric', 'Monday']
Entities: ['Azure', 'by Monday']
